In [40]:
import requests
import random
from random import randrange
import json
import nltk
from langdetect import detect
import os
import glob
from rdflib import *
from rdflib.graph import Graph
import re
import rdflib

In [41]:
#fonction pour écrire dans un fichier json
def write_json(chemin,contenu):
    with open(chemin,"w",encoding="utf-8") as w :
        w.write(json.dumps(contenu, indent=2, ensure_ascii=False))

In [42]:
#fonction pour récupérer le contenu d'un fichier json
def ouvrir_json(chemin) :
    with open(chemin,encoding="utf-8") as f :
        contenu = json.load(f)
    return contenu

In [43]:
#fonction pour extraire les lexèmes et les sens de Wiki Lexicographical Data avec une requête SPARQL
def requete_base(lang) : 
    langues = {"fr":"Q150","en" :"Q1860" }
    nom = "requetes_definitions_"+lang+".json"
    if os.path.exists("ressources/"+nom) :
        return nom
    requete_all = """
    SELECT DISTINCT ?l ?lemma ?sensestr WHERE {
       ?l dct:language wd:%s ;
            wikibase:lemma ?lemma;
            ontolex:sense/skos:definition ?sensestr.
        FILTER(LANG(?sensestr) = "%s")
        FILTER(LANG(?lemma) = "%s")
    }
    """%(langues[lang],lang,lang)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': requete_all, 'format': 'json'}).json()
    
    write_json(nom,data)
    return nom

In [44]:
#enregistrement des lexèmes et de leurs sens dans un dictionnaire
def liste_def(chemin) :

    resultats_json = ouvrir_json(chemin)
    liste_niv1 = resultats_json["results"]["bindings"]
    
    dico_resultats = {}
    identifiants = {}
    #identifiants : {lexeme_id : nom lexeme}
    #resultats : {lemme (str) : [sens1,sens2,...]}
    for chose in liste_niv1 :
        lemme = chose['lemma']['value']
        sens = chose['sensestr']['value'] 
        if lemme not in dico_resultats :
            dico_resultats[lemme] = []
            id_lexeme = chose['l']['value'].split('http://www.wikidata.org/entity/')[1]
            identifiants[id_lexeme] = lemme
        dico_resultats[lemme].append(sens)
       
    return dico_resultats,identifiants

In [45]:
def choix_reponse(dico_def,dico_id,dico_niv,lang,diff) :
    limite = 2
    estBon = False
    #entiteBonne == False
    #une reponse est bonne (estBon = True) si sa definition est en français et si elle comporte au moins 3 mots
    while estBon == False :
        cpt = 0
        #choix aleatoire d'une reponse, en prenant en compte la difficulte (id lexeme)
        lexeme_id = random.choice(dico_niv[diff])
        #on recupere la reponse
        rep_poss = dico_id[lexeme_id]
        #recuperer differents sens
        list_def = dico_def[rep_poss]
        #vérification de la validité de la réponse
        #limite : si au bout de plusieurs essais (on tente plusieurs definitions), on a toujours estBon = false
        #on choisit aleatoirement un autre lexeme
        while cpt < limite :
            #on choisit une definition aleatoirement (dans le cas des lexemes polysemiques)
            def_poss = random.choice(list_def)
            #ameliorer le test de langue si nécessaire
            #on vérifie que la définition est suffisamment longue (>=3) ; split rudimentaire pour le moment
            if len(def_poss.split()) >= 3 and detect(def_poss) == lang :
                reponse = rep_poss
                definition = def_poss
                break
            cpt+=1
        #résumé d'entités
        #resumes_entite(reponse,definition)
        
        #cela signifie que le mot n'a pas une définition satisfaisante
        if cpt == limite :
            estBon = False
        else :
            estBon = True
        
    return (reponse,definition)

In [46]:
#fonction permettant de transformer la réponse en indice pour l'utilisateur
def prep_rep(reponse,definition,diff) :
    cpt_crit = 35
    longueur = len(reponse)
    
    
     #selon le degre de difficulte, on augmente ou diminue le nombre d'indices
    if diff.lower() == 'f' or diff.lower() == 'tf':
        nb_lettres = round(longueur/2)
    if diff.lower() == 'm' :
        nb_lettres = round(longueur/3)
    if diff.lower() == 'd' or diff.lower() == 'td' :
        nb_lettres = round(longueur/4)
    #print(longueur)

    
    i = 0
    lettres = set()
    test_trait = False
    j = 0
    #on choisit les lettres (indice dans la chaine) qui seront affichees
    while i < nb_lettres and j < cpt_crit:
        #pour éviter les entrées répétées dans cette condition (cas du trait d'union)
        while test_trait == False :
            res = reponse.find("-")
            #si trait d'union alors ajouter aux lettres à afficher
            if res != -1 :
                lettres.add(res)
            #test_trait True pour sortir de la boucle
            test_trait = True
        #on ajoute une indice (lettre) à lettres (qui est un set, pour eviter les doublons)
        #avec set et le while, on verifie bien qu'on a des 25% de lettres
        lettres.add(randrange(longueur))
        i = len(lettres)
        j+=1
        
    indice = ""
    #char pour le set else _
    #trier la liste
    list_lettres = list(lettres)
    #print(list_lettres)
    k = 0
    ind = 0
    
    #affichage de _ ou de la lettre
    for ind,r in enumerate(reponse) :
        #and ind in list_lettres
        #k < len(list_lettres) and list_lettres[k] == ind
        if ind in list_lettres :
            indice+=r
            #print(r)
            #k+=1
        else :
            indice+=" _ "
        
        
    return indice

In [47]:
#creation du dictionnaire des scores
def pr_lexeme(fichier,identifiants) :
    g = Graph()
    g.parse(fichier, format="nt")
    dico_rangs = {}

    query = """
    SELECT DISTINCT ?s ?o
    WHERE {
        ?s ?p ?o.
    }
    """
    res = g.query(query)
    #print(res)
    cpt=0
    #dictionnaire : lexeme - score
    for i,r in enumerate(res) :
        lexeme = re.match(r'http://www.wikidata.org/entity/(L\d+)',f"{r.s}")
        if lexeme :
            cle = lexeme.group(1)
            if cle in identifiants :
                dico_rangs[cle] = round(float(f"{r.o}"),4)
    return dico_rangs


In [48]:
#fonction pour les niveaux de difficulté des jeux
def statistiques_rangs(dico) :
    longueur = len(dico)
    
    cinquieme = int(longueur/5)
    dico_trie = {}
    #tri du dictionnaire
    #https://waytolearnx.com/2019/04/comment-trier-un-dictionnaire-par-cle-ou-par-valeur-en-python.html#:~:text=Si%20nous%20voulons%20trier%20les,par%20ordre%20croissant%20par%20d%C3%A9faut).
    for k, v in sorted(dico.items(), key=lambda x: x[1]):
        dico_trie[k] = v
        
    #classement en niveaux des lexèmes
    dico_niveaux = {}
 
    liste_lexemes = list(dico_trie.keys())
  
    dico_niveaux['TD'] = liste_lexemes[:cinquieme]
    dico_niveaux['D'] = liste_lexemes[cinquieme:2*cinquieme]
    dico_niveaux['M'] = liste_lexemes[2*cinquieme:3*cinquieme]
    dico_niveaux['F'] = liste_lexemes[3*cinquieme:4*cinquieme]
    dico_niveaux['TF'] = liste_lexemes[4*cinquieme:]
    
    
    return dico_niveaux

In [55]:
def jeu() :
    #cont = "O"
    gain = 0
    total = 0 
    print("Bienvenue ! Devinez le mot recherché grâce à sa définition !")
    langue = input("Choisissez la langue : français (fr) ou anglais (en)")
    diff = input("Choisissez le niveau de difficulté : TF (très facile), F (facile) , M (moyen), D (difficile) ou TD (très difficile)")
    print()
    fichier = requete_base(langue)
    dico_def,dico_id = liste_def("ressources/"+fichier)
    
    
    if langue.lower() == "en" :
        dico_rangs = pr_lexeme("ressources/echantillon_score_PR_en.nt",dico_id.keys())
    if langue.lower() == "fr" :
        dico_rangs = pr_lexeme("ressources/score_PR_fr.nt",dico_id.keys())
    dico_niveaux = statistiques_rangs(dico_rangs)
    
    while total < 5 :
        reponse,definition = choix_reponse(dico_def,dico_id,dico_niveaux,langue.lower(),diff.upper())
        indice = prep_rep(reponse,definition,diff)
        print("Définition : ",definition)
        print("Indice",indice)
        rep_joueur = input("Que proposez-vous ?")
        if rep_joueur.lower() == reponse.lower() :
            if langue == "fr" :
                print("Bravo ! Bonne réponse !")
                print("-"*60)
            if langue == "en" :
                print("Good Answer !")
                print("-"*60)
            gain+=1
        else :
            if langue == "fr" :
                print("Non ! La réponse est %s"%reponse)
                print("-"*60)
            if langue == "en" :
                print("No ! The good answer is %s"%reponse)
                print("-"*60)
        total+=1
        #cont = input("Voulez-vous continuer à jouer ? (O / N)")
    print("Score : %d / %d"%(gain,total))

In [ ]:
jeu()